### Final Project SBA Loan Data
### DSC 630
### Sam Loyd
### November 2020
### PyCaret_Cat_Tuning

### This code was used for parameter tuning for the classification model for loan repayment.

In [1]:
# Required libraries
import pandas as pd
import numpy as np
# import matplotlib.pyplot as pt
import sklearn
from scipy.stats import pearsonr
import matplotlib.pyplot as matplotlib
from matplotlib.lines import Line2D
from matplotlib import pyplot
pd.options.display.float_format = '{:.2f}'.format
import seaborn 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from IPython.display import Image
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
# from pycaret.regression import *
from pycaret.classification import *

In [2]:
raw_sba_df = pd.read_csv("C:\\Users\\Loyd\\Documents\\630Project\\sbapython.csv")
raw_sba_df

,ApprovalFY,Term,NoEmp,CreateJob,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
0,35,84,4,0,60000,48000,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,35,60,2,0,40000,32000,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,35,180,7,0,287000,215250,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
3,35,60,2,0,35000,28000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,35,240,14,7,229000,229000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897105,35,60,6,0,70000,56000,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897106,35,60,6,0,85000,42500,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897107,35,108,26,0,300000,225000,0,1,0,0,...,0,0,0,0,1,1,1,1,0,1
897108,35,60,6,0,75000,60000,0,1,0,0,...,0,0,1,1,1,1,0,0,0,1


#### Normalize the data in case some models require it.  Non-gaussian so normalizing instead of standardizing.

In [3]:
# Normalize the data since its non-Gaussian
# scaler = MinMaxScaler() 
# sba_df[['Term','NoEmp','']] = scaler.fit_transform(data)
sba_df = raw_sba_df.copy()
sba_df[['Term','NoEmp','GrAppv','SBA_Appv']] = minmax_scale(sba_df[['Term','NoEmp','GrAppv','SBA_Appv']])

sba_df

,ApprovalFY,Term,NoEmp,CreateJob,GrAppv,SBA_Appv,NewExist_0,NewExist_1,NewExist_2,NewExist_U,...,Bank_4,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13
0,35,0.15,0.00,0,0.01,0.01,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,35,0.11,0.00,0,0.01,0.01,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,35,0.32,0.00,0,0.06,0.06,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
3,35,0.11,0.00,0,0.01,0.01,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,35,0.42,0.00,7,0.05,0.06,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897105,35,0.11,0.00,0,0.01,0.01,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897106,35,0.11,0.00,0,0.02,0.01,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
897107,35,0.19,0.00,0,0.06,0.06,0,1,0,0,...,0,0,0,0,1,1,1,1,0,1
897108,35,0.11,0.00,0,0.01,0.02,0,1,0,0,...,0,0,1,1,1,1,0,0,0,1


#### Split the data into 60,20,20 Train, Test Validate Sets.

In [4]:
# Create X from the features
feature_list = ['Term', 'NoEmp', 'GrAppv', 'SBA_Appv', 'NewExist_1', 'NewExist_2', 'UrbanRural_0', 'UrbanRural_1', 'UrbanRural_2', 'RevLineCr_2', 'RevLineCr_3', 'RevLineCr_4', 'FranchiseCode_6', 'FranchiseCode_8', 'FranchiseCode_10', 'FranchiseCode_11', 'FranchiseCode_12', 'LowDoc_3', 'LowDoc_4', 'NAICS_2', 'NAICS_3', 'NAICS_4', 'NAICS_5', 'NAICS_6', 'NAICS_7', 'NAICS_8', 'NAICS_9', 'NAICS_10', 'NAICS_11', 'Bank_2', 'Bank_3', 'Bank_4', 'Bank_5', 'Bank_6', 'Bank_7', 'Bank_8', 'Bank_9', 'Bank_10', 'Bank_11', 'Bank_12', 'Bank_13']

X = sba_df[(feature_list)]

# Create y from output
y = sba_df[['MIS_logical','CreateJob']]
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [5]:
# Now try with classification models for MIS_logical
X_train_caret_cat = X_train.copy()
X_train_caret_cat['MIS_logical'] = y_train['MIS_logical'].astype('bool')
X_train_caret_cat

,Term,NoEmp,GrAppv,SBA_Appv,NewExist_1,NewExist_2,UrbanRural_0,UrbanRural_1,UrbanRural_2,RevLineCr_2,...,Bank_5,Bank_6,Bank_7,Bank_8,Bank_9,Bank_10,Bank_11,Bank_12,Bank_13,MIS_logical
623129,0.11,0.00,0.02,0.02,1,0,1,0,0,0,...,0,0,0,0,0,1,1,0,0,True
589937,0.42,0.00,0.04,0.04,1,0,0,1,0,0,...,0,1,1,0,1,1,1,1,0,True
576584,0.49,0.00,0.21,0.21,1,0,0,1,0,0,...,0,1,0,1,0,0,1,1,0,True
618776,0.15,0.00,0.05,0.03,1,0,0,1,0,1,...,0,0,1,1,0,1,0,0,0,True
466287,0.21,0.00,0.05,0.06,1,0,1,0,0,0,...,0,1,1,1,1,1,1,1,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111316,0.21,0.00,0.15,0.15,0,1,1,0,0,0,...,1,1,1,1,1,0,0,0,1,True
883883,0.42,0.00,0.07,0.09,0,1,0,1,0,0,...,1,1,1,0,0,0,1,0,0,True
374049,0.14,0.00,0.01,0.01,1,0,0,1,0,0,...,0,1,1,0,1,1,1,1,0,True
54901,0.07,0.00,0.02,0.01,1,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,False


In [6]:
exp_clf_cat = setup(X_train_caret_cat, target='MIS_logical')

Setup Succesfully Completed!


,Description,Value
0,session_id,2923
1,Target Type,Binary
2,Label Encoded,"False: 0, True: 1"
3,Original Data,"(538266, 42)"
4,Missing Values,False
5,Numeric Features,4
6,Categorical Features,37
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [7]:
catmodel = create_model('catboost')


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9472,0.9770,0.9739,0.9624,0.9682,0.8138,0.8143
1,0.9464,0.9770,0.9740,0.9614,0.9677,0.8105,0.8111
2,0.9461,0.9769,0.9733,0.9617,0.9675,0.8098,0.8103
3,0.9439,0.9752,0.9710,0.9614,0.9661,0.8029,0.8032
4,0.9443,0.9766,0.9723,0.9606,0.9664,0.8035,0.8041
5,0.9434,0.9745,0.9716,0.9602,0.9659,0.8004,0.8009
6,0.9455,0.9775,0.9723,0.9620,0.9671,0.8082,0.8086
7,0.9453,0.9771,0.9718,0.9622,0.9670,0.8077,0.8080
8,0.9460,0.9777,0.9741,0.9609,0.9675,0.8090,0.8096
9,0.9461,0.9785,0.9737,0.9615,0.9675,0.8098,0.8104


In [8]:
tuned_catmod_custom = tune_model(catmodel)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9456,0.9752,0.9742,0.9604,0.9672,0.8073,0.8080
1,0.9444,0.9752,0.9740,0.9591,0.9665,0.8025,0.8033
2,0.9450,0.9747,0.9743,0.9595,0.9669,0.8047,0.8055
3,0.9425,0.9735,0.9714,0.9594,0.9654,0.7972,0.7977
4,0.9431,0.9749,0.9738,0.9579,0.9658,0.7977,0.7986
5,0.9422,0.9728,0.9724,0.9581,0.9652,0.7950,0.7957
6,0.9437,0.9755,0.9723,0.9599,0.9661,0.8011,0.8017
7,0.9420,0.9752,0.9715,0.9586,0.9650,0.7948,0.7954
8,0.9442,0.9757,0.9741,0.9588,0.9664,0.8017,0.8026
9,0.9446,0.9765,0.9739,0.9594,0.9666,0.8033,0.8041


In [10]:
# Default tuning
tuned_catmod_custom